In [ ]:
# MODELING
# --------------------------------------------------------------------------------------

# Select modeling techniques: Determine which algorithms to try (e.g. regression, neural
# net).
# 
# Generate test design: Pending your modeling approach, you might need to split the data
# into training, test, and validation sets.
# 
# Build model: As glamorous as this might sound, this might just be executing a few
# lines of code like “reg = LinearRegression().fit(X, y)”.
# 
# Assess model: Generally, multiple models are competing against each other, and the
# data scientist needs to interpret the model results based on domain knowledge, the
# pre-defined success criteria, and the test design.

In [ ]:
# CONCLUSIONS
# --------------------------------------------------------------------------------------

# - ...

In [1]:
import os
import shutil

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as sp
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

pd.set_option("display.max_columns", None)
YEAR = "2021"

shutil.rmtree(f"../output/d-modeling")
os.makedirs(f"../output/d-modeling", exist_ok=True)
os.makedirs(f"../data/d-modeling", exist_ok=True)

file = f"../data/c-data-preparation/{YEAR}-raw.csv"
raw = pd.read_csv(file)

file = f"../data/c-data-preparation/{YEAR}-preprocessed.csv"
preprocessed = pd.read_csv(file)

pd.set_option("display.expand_frame_repr", False)

In [2]:
print(raw.head())

    dia_semana  uf     br     km              municipio                                     causa_acidente              tipo_acidente classificacao_acidente     fase_dia  sentido_via condicao_metereologica tipo_pista    tracado_via uso_solo  pessoas  mortos  feridos_leves  feridos_graves  ilesos  ignorados  feridos  veiculos   latitude  longitude regional delegacia             uop   ano  mes  dia  hora  minuto
0  sexta-feira  PR  277.0   51.3   SAO JOSE DOS PINHAIS                                 Pista Escorregadia  Saída de leito carroçável    Com Vítimas Feridas    Pleno dia  Decrescente         Garoa/Chuvisco      Dupla          Curva      Não        4       0              1               0       3          0        1         3 -25.595160 -48.907008  SPRF-PR  DEL01-PR  UOP05-DEL01-PR  2021    1    1    15      45
1  sexta-feira  SC  470.0   79.1                INDAIAL                             Transitar na contramão            Colisão frontal     Com Vítimas Fatais    Pleno dia   

In [3]:
print(preprocessed.head())

   dia_semana   uf        br        km  municipio  causa_acidente  tipo_acidente  classificacao_acidente  fase_dia  sentido_via  condicao_metereologica  tipo_pista  tracado_via  uso_solo   pessoas    mortos  feridos_leves  feridos_graves    ilesos  ignorados   feridos  veiculos  latitude  longitude  regional  delegacia       uop  ano  mes       dia      hora    minuto
0    0.666667  0.0  0.214286  0.070178   0.842825        0.735294         0.9375                     0.5  1.000000          1.0                0.222222         0.0     0.000000       0.0  0.056604  0.000000       0.029412        0.000000  0.065217   0.000000  0.027778  0.181818  0.741768   0.955800       0.0   0.000000  0.956044  0.0  0.0  0.000000  0.652174  0.762712
1    0.666667  1.0  0.785714  0.108208   0.412301        0.941176         0.2500                     0.0  1.000000          0.0                0.777778         1.0     0.555556       0.0  0.150943  0.105263       0.147059        0.066667  0.000000   0.166667

In [7]:
# Plotting clusters on a map
# --------------------------------------------------------------------------------------

def plot(df, labels, dst, title):
    num_clusters = len(np.unique(labels))
    color_scale = ["hsl(" + str(h) + ",50%" + ",50%)" for h in np.linspace(0, 360, num_clusters)]

    fig = go.Figure()

    for cluster_label, color in zip(np.unique(labels), color_scale):
        # Skip noise
        if cluster_label == -1:
            continue

        cluster_mask = labels == cluster_label
        cluster_data = df[cluster_mask]

        scatter = go.Scattermapbox(
            lat=cluster_data.latitude,
            lon=cluster_data.longitude,
            mode="markers",
            marker=dict(size=7.5, color=color),
            # text=f"({cluster_label}) {cluster_data.causa_acidente}",
            text=f"{cluster_label}",
            showlegend=False,
        )

        fig.add_trace(scatter)

    fig.update_layout(
        title=title,
        title_x=0.95,
        title_y=0.1,
        mapbox_style="open-street-map",
        mapbox=dict(center=dict(lat=-28, lon=-52), zoom=5.5),
        margin={"l": 0, "r": 0, "t": 0, "b": 0},
    )

    # fig.show()
    fig.write_html(f"{dst}.html")

In [ ]:
# Elbow method to determine the optimal number of clusters
# --------------------------------------------------------------------------------------

length = len(preprocessed)
kmax = int(np.sqrt(length))
kmin = 2
ks = range(kmin, kmax)
columns = ["latitude", "longitude"]
X = preprocessed[columns]

inertia = []
silhouette = []

for i, k in enumerate(ks):
    print(f"[{i + 1}] n_clusters: {k}")

    model = KMeans(n_clusters=k, random_state=42)
    model.fit(X)
    
    inertia.append(model.inertia_)
    silhouette.append(silhouette_score(X, model.labels_))

In [ ]:
# Elbow method to determine the optimal number of clusters (plot)
# --------------------------------------------------------------------------------------

# Create subplots using make_subplots
titles = ("Inertia vs Number of Clusters", "Silhouette Score vs Number of Clusters")
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=titles)

# Plotting inertia values
trace = go.Scatter(x=list(ks), y=inertia, mode="lines+markers", name="Inertia")
fig.add_trace(trace, row=1, col=1)

# Plotting silhouette scores
trace = go.Scatter(x=list(ks), y=silhouette, mode="lines+markers", name="Silhouette Score")
fig.add_trace(trace, row=1, col=2)

fig.update_xaxes(title_text="Number of Clusters", row=1, col=1)
fig.update_yaxes(title_text="Inertia", row=1, col=1)
fig.update_xaxes(title_text="Number of Clusters", row=1, col=2)
fig.update_yaxes(title_text="Silhouette Score", row=1, col=2)
fig.update_layout(title="K-Means Clustering Evaluation: Elbow Method", showlegend=False)

fig.show()

In [8]:
# Build top 3 models according to silhouette score
# --------------------------------------------------------------------------------------

ks = [48, 57, 70]
for i, k in enumerate(ks):
    header = f"k: {k}"
    print(header)
    print("-" * len(header))
    print(f"Columns: {columns}")

    X = preprocessed[columns].values

    model = KMeans(n_clusters=k, random_state=42)
    model.fit(X)

    # Number of clusters in labels
    labels = model.labels_
    score = metrics.silhouette_score(X, labels)

    dst = f"../output/d-modeling/{YEAR}-kmeans-{k}-{i}"
    title = (
        f"k: {k}, score: {score:.3f}<br>"
        f"columns: {columns}"
    )

    print(f"Silhouette coefficient: {score:.3f}\n")
    plot(raw, labels, dst, title)

k: 48
-----
Columns: ['latitude', 'longitude']
Silhouette coefficient: 0.572

k: 57
-----
Columns: ['latitude', 'longitude']
Silhouette coefficient: 0.575

k: 70
-----
Columns: ['latitude', 'longitude']
Silhouette coefficient: 0.579



In [11]:
# Build final model: regioes
# --------------------------------------------------------------------------------------

k = 48
columns = ["latitude", "longitude"]

header = f"k: {k}"
print(header)
print("-" * len(header))
print(f"Columns: {columns}")

X = preprocessed[columns].values

model = KMeans(n_clusters=k, random_state=42)
model.fit(X)

# Number of clusters in labels
labels = model.labels_
score = metrics.silhouette_score(X, labels)

print(f"Silhouette coefficient: {score:.3f}\n")

# Save dataset with cluster column to csv
# --------------------------------------------------------------------------------------

raw["cluster"] = model.labels_
raw.to_csv(f"../data/d-modeling/{YEAR}-regioes.csv", index=False)

k: 48
-----
Columns: ['latitude', 'longitude']
Silhouette coefficient: 0.572

